First we import the regular expressions module. Read in a GenBank file using the read method, not readlines.

In [84]:
import re

my_file = open("DMD.gb")

gbFile = my_file.read()

print(gbFile)

LOCUS       NG_012232            2227382 bp    DNA     linear   PRI 15-AUG-2020
DEFINITION  Homo sapiens dystrophin (DMD), RefSeqGene (LRG_199) on chromosome
            X.
ACCESSION   NG_012232
VERSION     NG_012232.1
KEYWORDS    RefSeq; RefSeqGene.
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (bases 1 to 2227382)
  AUTHORS   Achermann,J.C. and Vilain,E.J.
  TITLE     NR0B1-Related Adrenal Hypoplasia Congenita
  JOURNAL   (in) Adam MP, Ardinger HH, Pagon RA, Wallace SE, Bean LJH, Stephens
            K and Amemiya A (Eds.);
            GENEREVIEWS((R));
            (1993)
   PUBMED   20301604
REFERENCE   2  (bases 1 to 2227382)
  AUTHORS   Hershberger,R.E. and Morales,A.
  TITLE     Dilated Cardiomyopathy Overview
  JOURNAL   (in) Adam MP, Ardinger HH, Pagon RA, Wallac

Use the string find method to identify the location in the string of the beginning and end of the section containing the DNA sequence. The code below extracts the substring with the DNA sequence using the tags that enclose this section. White space and numbers are eliminated using the string replace function. The first use of regular expressions (re.sub) is used to find and delete all digits. The first part of the sequence is printed out to confirm the DNA sequence is now an uninterrupted string.

In [85]:
origin = gbFile.find("ORIGIN")
end  = gbFile.find("//")
dnaSequence = gbFile[origin+5:end]

# my first attempt at this problem 
#oupt = ""
#for c in gbFile[origin+5:]:
#    
#    if c == "a" or c == "t" or c == "c" or c == "g":
#        oupt += c
#    oupt.replace("\n","")
#    "".join(oupt)
#    print(oupt)


In [86]:
count_A = dnaSequence.count("a")
print (count_A)

693847


Now we use the full power of regular expressions to identify the coding segments. It takes time to master regular expression, but you should understand this pattern fimnds the section containing the coordinates of the coding sequences (CDS) in the GenBank file.

We first get the pattern into memory to make for more efficient searching by first compiling it.
Here is the meaning of each part of the pattern we are looking for. You should load the GenBank file into your text editor to find this section.

(\s){5} looks for the beginning of this section with 5 spaces (CDS) looks for the string "CDS" (\s)* looks for zero or more white spaces (join) looks for the string "join" (() is an example of looking for "(", which must be escaped with a backslash to indicate it is part of the string pattern. (()(((\d)(..)(\d)(,))(\s))(\d)(..)(\d)()) is the most complex pattern--which describes the set of coordinate pairs. The set of coordinates are all enclosed within parentheses, and each coordinate is separated from its partner by "..".
((\d)(..)(\d)(,))(\s)) looks for the first to next to last lines (\d)(..)(\d)() looks for the unique last line, including the final ")"

In [87]:
remove = ["\n", "\t", " ", "ORIGIN", "N"]
for i in remove:
    dnaSequence = dnaSequence.replace(i,"")

dnaSequence = re.sub("\d", "", dnaSequence) #\d is digits 
print(dnaSequence[:1000])
print(len(dnaSequence))

cattttatgaagccagaatcacactggcaccaaagccaggcaaagataccacaataaaagaacactacaggagggtaactctgatgtgtatagatccaaaaatcctcaataaaatactagcaaaacaaattcaacagtacatcaaaaggattatataccatgatcaggtgggatttatctccaagatgcaaggttggtatggcatgcaaatcaatcaatgtgatatatcacataaacggagtaaaagacaagaaccacatgaccatcttaatagatgcagaaaaaaccatttgaaaaagttcaacatgtattcatgggtaaattgaaaactgtcaacaaaataagtgtagaatgatcttgcatcaacataataaaggccatttatgaaaaactcatagctaacattgtaaccaacggggaaaaaattgaaagcttttcctctaagatctggtacaatgcagggattcccatgatcaacacttctgttcaacatagtattggaagtcctagtcagagcaattaggcaagagaaagaaataaaaggcagtggaattggaaaggaagaagtaaaattatctgtttgtagatgccacgattctatatgtagaaagccctaatgatttcattaaaaatttgtcagaactaatcaatgaattcagtaaaattgctagatacaaaatcaacatacaaatattggtagcatttcttttctctaaaaatgaaatcagttgatttcattttgaaaaggaaatcaataaaatgtttagatttagctttttattgatacattacacatttatggggtacatgtgatattccaatatatacatacaatgtgtaatgatcgaatcaaggtgattaggatgtccatcatctcaaatatttatcatttctttgtaatgggaacattccaaatcttctcttctggcaatttgaaatatactataaattcatgttaactatagctgccctactatgctctcaaacactagaatttatt

In [88]:
pattern = re.compile('(\s){5}(CDS)(\s)*(join)(\()(((\d)*(..)(\d)*(,))*(\s)*)*(\d)*(..)(\d)*(\))')

We now use the regular expression search function to find and retrive the specific string that matches the pattern above. We retrieve this found string use the group() method from the Match object generated in this search.

In [89]:
CDS_list = re.search(pattern,gbFile)
print(CDS_list)
CDS_list = CDS_list.group()
print(CDS_list)
remove = ["join(", "CDS", ")", "\n", "\t", " "]
for i in remove:
    CDS_list = CDS_list.replace(i,"")


CDS_list = CDS_list.split(",")
exons_coords= []
#print(CDS_list)
for i in CDS_list:
    s = i.split("..")
    exons_coords.append((int(s[0]),int(s[1])))

print(exons_coords)


<re.Match object; span=(8472, 10335), match='     CDS             join(133298..133328,324410..>
     CDS             join(133298..133328,324410..324471,494790..494882,
                     499750..499827,521223..521315,527970..528142,
                     534999..535117,645317..645498,646612..646740,
                     699458..699646,700297..700478,730157..730307,
                     748734..748853,770764..770865,770973..771080,
                     778729..778908,799276..799451,826479..826602,
                     842768..842855,853092..853333,859511..859691,
                     872301..872446,875900..876112,879911..880024,
                     881016..881171,889778..889948,895972..896154,
                     903296..903430,906220..906369,932697..932858,
                     954429..954539,954936..955109,958145..958300,
                     963930..964100,979411..979590,979900..980028,
                     981652..981822,996082..996204,998530..998667,
                     1001324

CDS_list is now a string containing the coordinates we need to get indices of the coding segments. We remove everything from the string except coordinates, which are separated by commas.

Now convert this string to a list using the string.split method, the separates the coordinates into elements of the list using the commas as delimiters. A simple loop converts the elements from being strings to integers, which we need to extract substrings.

Successive pairs of coordinates are used to extract the coding portions of the DNA sequence and stich them together into one string. Coordinate pairs are converted to ranges for substrings. Each substring is append to the initiallt enm The string represents DNA version of the coding portion of the messenger RNA (minus its 5' and 3' UTRs). If successful, the string should start with the ATG start codon and end with a stop codon. To use the genetic code dictionary, the sequence is converted to upper case.

In [97]:
from traitlets import Integer


DNAsequence = ""
for i in exons_coords:
    x = i[0] -1
    y = i[1]
    #print(x)
    #print(y)
    DNAsequence += dnaSequence[x:y]


print(DNAsequence)

atgctttggtgggaagaagtagaggactgttatgaaagagaagatgttcaaaagaaaacattcacaaaatgggtaaatgcacaattttctaagtttgggaagcagcatattgagaacctcttcagtgacctacaggatgggaggcgcctcctagacctcctcgaaggcctgacagggcaaaaactgccaaaagaaaaaggatccacaagagttcatgccctgaacaatgtcaacaaggcactgcgggttttgcagaacaataatgttgatttagtgaatattggaagtactgacatcgtagatggaaatcataaactgactcttggtttgatttggaatataatcctccactggcaggtcaaaaatgtaatgaaaaatatcatggctggattgcaacaaaccaacagtgaaaagattctcctgagctgggtccgacaatcaactcgtaattatccacaggttaatgtaatcaacttcaccaccagctggtctgatggcctggctttgaatgctctcatccatagtcataggccagacctatttgactggaatagtgtggtttgccagcagtcagccacacaacgactggaacatgcattcaacatcgccagatatcaattaggcatagagaaactactcgatcctgaagatgttgataccacctatccagataagaagtccatcttaatgtacatcacatcactcttccaagttttgcctcaacaagtgagcattgaagccatccaggaagtggaaatgttgccaaggccacctaaagtgactaaagaagaacattttcagttacatcatcaaatgcactattctcaacagatcacggtcagtctagcacagggatatgagagaacttcttcccctaagcctcgattcaagagctatgcctacacacaggctgcttatgtcaccacctctgaccctacacggagcccatttccttcacagcatttggaagctcctgaagacaagtcatttggcagttcat

Genetic code dictionary, DNA version

In [91]:
gencode = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'*', 'TAG':'*',
    'TGC':'C', 'TGT':'C', 'TGA':'*', 'TGG':'W'}

def DNAtoAA(s,upper = True, joiner = "", outputUpper = True):
        """Turns DNA sequnce into the Ameno Acid sequence
        s: input sequence
        upper: tells function wheather input string is in upper case or lowercase
        joiner: tells the function what string it should use to join the ameno acids 
        outputUpper: Tells the funciton if the output should be capitalised"""
        if not upper:
            s = s.upper()
        dic =  {
                'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
                'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
                'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
                'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
                'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
                'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
                'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
                'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
                'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
                'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
                'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
                'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
                'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
                'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
                'TAC':'Y', 'TAT':'Y', 'TAA':'*', 'TAG':'*',
                'TGC':'C', 'TGT':'C', 'TGA':'*', 'TGG':'W'}
        oupt = []
        i1 = 0 
        while i1+3 <= len(s):
            key = s[i1:i1+3]
            AA =    dic[key]   
            if not outputUpper:
                aa = AA[0].lower()
                AA = aa+AA[1:]
            oupt.append(AA)
            i1 = i1 + 3
            
        oupt = joiner.join(oupt)
        return(oupt)



Now translate the open reading frame

In [92]:
AAsequence = DNAtoAA(DNAsequence,upper = False, joiner = "", outputUpper = True)

print(AAsequence)

MLWWEEVEDCYEREDVQKKTFTKWVNAQFSKFGKQHIENLFSDLQDGRRLLDLLEGLTGQKLPKEKGSTRVHALNNVNKALRVLQNNNVDLVNIGSTDIVDGNHKLTLGLIWNIILHWQVKNVMKNIMAGLQQTNSEKILLSWVRQSTRNYPQVNVINFTTSWSDGLALNALIHSHRPDLFDWNSVVCQQSATQRLEHAFNIARYQLGIEKLLDPEDVDTTYPDKKSILMYITSLFQVLPQQVSIEAIQEVEMLPRPPKVTKEEHFQLHHQMHYSQQITVSLAQGYERTSSPKPRFKSYAYTQAAYVTTSDPTRSPFPSQHLEAPEDKSFGSSLMESEVNLDRYQTALEEVLSWLLSAEDTLQAQGEISNDVEVVKDQFHTHEGYMMDLTAHQGRVGNILQLGSKLIGTGKLSEDEETEVQEQMNLLNSRWECLRVASMEKQSNLHRVLMDLQNQKLKELNDWLTKTEERTRKMEEEPLGPDLEDLKRQVQQHKVLQEDLEQEQVRVNSLTHMVVVVDESSGDHATAALEEQLKVLGDRWANICRWTEDRWVLLQDILLKWQRLTEEQCLFSAWLSEKEDAVNKIHTTGFKDQNEMLSSLQKLAVLKADLEKKKQSMGKLYSLKQDLLSTLKNKSVTQKTEAWLDNFARCWDNLVQKLEKSTAQISQAVTTTQPSLTQTTVMETVTTVTTREQILVKHAQEELPPPPPQKKRQITVDSEIRKRLDVDITELHSWITRSEAVLQSPEFAIFRKEGNFSDLKEKVNAIEREKAEKFRKLQDASRSAQALVEQMVNEGVNADSIKQASEQLNSRWIEFCQLLSERLNWLEYQNNIIAFYNQLQQLEQMTTTAENWLKIQPTTPSEPTAIKSQLKICKDEVNRLSGLQPQIERLKIQSIALKEKGQGPMFLDADFVAFTNHFKQVFSDVQAREKELQTIFDTLPPMRYQETMSAIRTWVQQSETKLSIPQLSVTDYEIMEQRLGELQALQSSLQEQQSGLYYLS

In [94]:
pattern2 = re.compile('C.{2}C.{4,8}[RHDGSCV][YWFMVIL].[CS].{2,5}[CHEQ].[DNSAGE][YFVLI].[LIVFM]C.{2}C')
AA_sub = re.search(pattern2,AAsequence)
print(AA_sub)
AA_sub = AA_sub .group()
print(AA_sub)

<re.Match object; span=(3312, 3340), match='CNICKECPIIGFRYRSLKHFNYDICQSC'>
CNICKECPIIGFRYRSLKHFNYDICQSC
